In [34]:

from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
import cv2
from tqdm import tqdm
import os
from glob import glob
import pandas as pd
import json
from skimage.draw import polygon2mask
import shutil

In [84]:
json_list=glob('../../data/origin_json/BRDC/*.json')
image_path="../../data/origin_json/BRDC/"
save_img_path='../../data/area_segmentation/BRDC/image/'
save_mask_path='../../data/area_segmentation/BRDC/mask/'

In [88]:
class_list=['NT_epithelial', 'NT_immune', 'NT_stroma', 'TP_in_situ', 'TP_invasive']
for j in tqdm(range(len(json_list))):
    with open(json_list[j], 'r') as f:

        json_data = json.load(f)
    anno=json_data['content']['file']['objects']
    file_name=json_data['content']['file']['filename']
    # for i in range(len(anno)):
    #     class_list.append(anno[i]['label_nm'])
    # class_list=list(set(class_list))
    shutil.copyfile(image_path+file_name, save_img_path+file_name)
    mask=np.zeros((1024,1024,len(class_list)))
    for k in range(len(anno)):
        coordinate=np.array(anno[k]['coordinate'])
        class_nm=anno[k]['label_nm']
        mask_temp=polygon2mask((1024,1024), coordinate)
        mask[...,class_list.index(class_nm)]+=mask_temp*255
    mask=mask.astype(np.uint8)
    np.save(save_mask_path+file_name.replace('.jpeg', ''), mask) # x_save.npy
    

100%|██████████| 562/562 [04:30<00:00,  2.07it/s]


In [87]:
json_data['content']['file']['filename'].replace('.jpeg', '')

'NIA6_R_BRDC_BRCA-YN-00370-S-TP-01_27_28'